## Assignment - 2
### Kaustav Vats (2016048)

In [18]:
import codecs
import string
from nltk.tokenize import sent_tokenize, word_tokenize
import re, os
import pickle
from matplotlib import pyplot as plt
from random import randint as randi
from math import log10
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [19]:
def GetAllTokens(fileNameList, FolderPath):
    tokens = []
    for fileName in fileNameList:
        file = open(FolderPath + fileName, 'r')
        text = pre_processing(file.read())
        tokens += word_tokenize(text)
        file.close()
    return tokens

def GetTokens(folderName):
    folderPath = "20_newsgroups/" + folderName + "/"
    All_folders = os.listdir(folderPath)
    TestFolderCount = int(len(All_folders) * 0.3)
    Test_Tokens = []
    Train_Tokens = []

    for i in range(TestFolderCount):
        index = randi(0, len(All_folders))
        doc = All_folders.pop(index)
        file = open(folderPath + doc, 'r')
        text = pre_processing(file.read())
        temp = word_tokenize(text)
        Test_Tokens += temp
        file.close()
        
    for doc in All_folders:
        file = open(folderPath + doc, 'r')
        text = pre_processing(file.read())
        temp = word_tokenize(text)
        Train_Tokens += temp
        file.close()
        
    return Train_Tokens, Test_Tokens, 
    
def pre_processing(text):
    header = text.find("\n\n")
    text = text[header:]
    text = text.strip()
    text = text.lower()
    text = re.sub(r'[^\w]', ' ', text) # Word Preprocessing
    return text

def BagOfWords(tokens):
    bof = {}
    for t in tokens:
        if (t in bof):
            bof[t] += 1
        else:
            bof[t] = 1
    return bof

def TrainTestSplit(folderName):
    folderPath = "20_newsgroups/" + folderName + "/"
    All_folders = os.listdir(folderPath)
    X1, X2 = train_test_split(All_folders, test_size=0.3)
    return X1, X2

In [20]:
# Main Section
# 2 Class

# Folder Split
mTrain, mTest = TrainTestSplit("rec.motorcycles")
bTrain, bTest = TrainTestSplit("rec.sport.baseball")


# For Training on Full Dataset
mTrain = mTrain + mTest
bTrain = bTrain + bTest

mTokens = GetAllTokens(mTrain, "20_newsgroups/rec.motorcycles/")
bTokens = GetAllTokens(bTrain, "20_newsgroups/rec.sport.baseball/")

bow_m = BagOfWords(mTokens)
bow_b = BagOfWords(bTokens)
Vocab = list(set(list(bow_m.keys()) + list(bow_b.keys())))
VocabDic = {}
for i in range(len(Vocab)):
    VocabDic[Vocab[i]] = i
print(len(bow_m))
print(len(bow_b))
print(len(Vocab))

14709
13204
22401


In [21]:

def Fit(K=1):
    if (os.path.isfile("Likli_Proba_2_{}".format(K))):
        LiklihoodProba = np.load("Likli_Proba_2_{}".format(K))
        return LiklihoodProba
    LiklihoodProba = np.zeros((len(Vocab), 2))
    for i in range(len(Vocab)):
        word = Vocab[i]
        if(word in bow_m):
            wc = bow_m[word] + K
            den = len(mTokens) + K*len(Vocab)
            LiklihoodProba[i, 0] = wc/den
        else:
            wc = K
            den = len(mTokens) + K*len(Vocab)
            LiklihoodProba[i, 0] = wc/den
        if(word in bow_b):
            wc = bow_b[word] + K
            den = len(bTokens) + K*len(Vocab)
            LiklihoodProba[i, 1] = wc/den
        else:
            wc = K
            den = len(bTokens) + K*len(Vocab)
            LiklihoodProba[i, 1] = wc/den
    np.save("Likli_Proba_2_{}".format(K), LiklihoodProba)
    return LiklihoodProba
            
            
Priors = [len(mTrain)/(len(mTrain)+len(bTrain)), len(bTrain)/(len(mTrain)+len(bTrain))]
print(Priors)
def Predict(Doc, LiklihoodProba, K=1):
    file = open(Doc, 'r')
    text = pre_processing(file.read())
    tokens = word_tokenize(text)
    file.close()
    
    probs = [log10(Priors[0]), log10(Priors[1])]
    # Class 0 & 1
    for i in range(len(tokens)):
        if (tokens[i] in Vocab):
            probs[0] += log10(LiklihoodProba[VocabDic[tokens[i]], 0])
            probs[1] += log10(LiklihoodProba[VocabDic[tokens[i]], 1])
        else:
            probs[0] += log10(K/(len(mTokens) + K*len(Vocab)))
            probs[1] += log10(K/(len(bTokens) + K*len(Vocab)))
    if (probs[0] > probs[1]):
        return 0
    else:
        return 1


[0.5, 0.5]


In [22]:
for k in [1, 5, 10, 100]:
    LiklihoodProba = Fit(K=k)

In [23]:
for k in [1, 5, 10, 100]:
    LiklihoodProba = Fit(K=k)
    correct = 0
    for i in tqdm(range(len(mTest))):
    #     print(mTest[i])
        c = Predict("20_newsgroups/rec.motorcycles/" + mTest[i], LiklihoodProba, K=k)
        if (c == 0):
            correct+=1
    print(correct)
    for i in tqdm(range(len(bTest))):
        c = Predict("20_newsgroups/rec.sport.baseball/"+bTest[i], LiklihoodProba, K=k)
        if (c == 1):
            correct+=1
    print("Accuracy[K={}]: ".format(k), correct/(len(mTest)+len(bTest)))


300



Accuracy[K=1]:  0.9983333333333333



299



Accuracy[K=5]:  0.995



295



Accuracy[K=10]:  0.9883333333333333



241



Accuracy[K=100]:  0.8983333333333333


## Testing Part

In [ ]:
text = str(input())